In [83]:
import pandas as pd
import re
from PyKakao import Local
from pytimekr import pytimekr
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import catboost as cb

import warnings
warnings.filterwarnings('ignore')

In [93]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")
train = train[test.columns.tolist() + ['사망자수', '중상자수',
       '경상자수', '부상자수', 'ECLO']]

accident = pd.read_csv("./data/external_open/countrywide_accident.csv")
CCTV = pd.read_csv("./data/external_open/대구CCTV정보.csv", encoding='cp949')
light = pd.read_csv("./data/external_open/대구보안등정보.csv", encoding='cp949')
child = pd.read_csv("./data/external_open/대구어린이보호구역정보.csv", encoding='cp949')

In [94]:
CCTV = CCTV[["위도","경도","소재지지번주소", "도로노선방향", "도로종류", "단속구분", "제한속도", "단속구간위치구분", "과속단속구간길이", "보호구역구분", "설치연도"]]
light = light[["위도","경도", "소재지지번주소","설치개수", "설치연도", "설치형태"]]
child = child[["위도","경도","소재지지번주소", "시설종류", "CCTV설치여부","CCTV설치대수","보호구역도로폭"]]

In [95]:
# ID 삭제
train = train.drop(["ID"], axis = 1)
test = test.drop(["ID"], axis = 1)

# Hour 변수 추가
train["Hour"] = [i.split()[1] for i in train.사고일시]
train["사고일시"] = [i.split()[0] for i in train.사고일시]
test["Hour"] = [i.split()[1] for i in test.사고일시]
test["사고일시"] = [i.split()[0] for i in test.사고일시]

# Year, Month, Day 변수 추가
train["Year"] = [i.split("-")[0] for i in train.사고일시]
train["Month"] = [i.split("-")[1] for i in train.사고일시]
train["Day"] = [i.split("-")[2] for i in train.사고일시]
train["Year_Month"] = [train.Year[i] + train.Month[i] for i in range(len(train))]
train = train.drop(["사고일시"], axis = 1)
test["Year"] = [i.split("-")[0] for i in test.사고일시]
test["Month"] = [i.split("-")[1] for i in test.사고일시]
test["Day"] = [i.split("-")[2] for i in test.사고일시]
test["Year_Month"] = [test.Year[i] + test.Month[i] for i in range(len(test))]
test = test.drop(["사고일시"], axis = 1)

# 시군구 분리
train["군"] = [i.split(" ")[1] for i in train.시군구]
train["구"] = [i.split(" ")[2] for i in train.시군구]
train = train.drop(["시군구"], axis = 1)
test["군"] = [i.split(" ")[1] for i in test.시군구]
test["구"] = [i.split(" ")[2] for i in test.시군구]
test = test.drop(["시군구"], axis = 1)


# 도로형태 분리
train["도로형태대분류"] = [i.split("-")[0] for i in train.도로형태]
train["도로형태소분류"] = [i.split("-")[1] for i in train.도로형태]
train = train.drop(["도로형태"], axis = 1)
test["도로형태대분류"] = [i.split("-")[0] for i in test.도로형태]
test["도로형태소분류"] = [i.split("-")[1] for i in test.도로형태]
test = test.drop(["도로형태"], axis = 1)

# CCTV 데이터 가공
# CCTV가 있는 소재지의 구만 추출
CCTV["구"] = [i.split()[2] if type(i) == str else i for i in CCTV.소재지지번주소]
LOCAL = Local(service_key = "3206f186fc6467db13a17a82c6e3e668")
for i in CCTV[CCTV.구.isna()].index:
    CCTV.구.iloc[i] = LOCAL.geo_coord2address(CCTV.iloc[i].경도, CCTV.iloc[i].위도, dataframe=True).address[0]["region_3depth_name"]
CCTV.구 = [re.sub(r"동[0-9]+-[0-9]+$|[0-9].[0-9]동$|[0-9]동$", '동', i) for i in CCTV.구]
CCTV.구 = [re.sub(r"가[0-9]+-[0-9]+$", '가', i) for i in CCTV.구]

# train 데이터의 사고가 난 시군구 노선방향, 도로종류, 단속구간위치구분, 과속단속구간길이는 
# 보호구역 여부
CCTV["어린이보호구역"] = [1 if i == 2 else 0 for i in CCTV.보호구역구분]
CCTV["노인보호구역"] = [1 if i == 1 else 0 for i in CCTV.보호구역구분]
CCTV["기타보호구역"] = [1 if i == 99 else 0 for i in CCTV.보호구역구분]

# 단속구분 -> 데이터 확인 결과 통행위반인 경우는 없으므로 삭제
CCTV["속도단속"] = [1 if i == 1 else 0 for i in CCTV.단속구분]
CCTV["신호단속"] = [1 if i == 2 else 0 for i in CCTV.단속구분]
CCTV["불법주정차"] = [1 if i == 4 else 0 for i in CCTV.단속구분]
CCTV["기타단속"] = [1 if i == 99 else 0 for i in CCTV.단속구분]

CCTV = CCTV[['설치연도', '구', '어린이보호구역', '노인보호구역', '기타보호구역', '속도단속', '신호단속', 
       '불법주정차', '기타단속']]

# 사고 날짜는 정확한데 CCTV 설치날짜는 세세하지않고 연도만 나와있음
# train 데이터는 2019년의 경우 2018년 이전 CCTV 갯수만 반영, 2020년의 경우 2019년 이전 CCTV 갯수만 반영한다.
def df_add(row, df2):
    if row.구 in df2.구.tolist():
        row[row.index.drop(["구"])] += df2.iloc[df2.구.tolist().index(row.구)][df2.columns.drop(["구"])]
    return row

CCTV_2018_before = CCTV[CCTV.설치연도 < 2019]
CCTV_2019 = CCTV[CCTV.설치연도 == 2019]
CCTV_2020 = CCTV[CCTV.설치연도 == 2020]
CCTV_2021 = CCTV[CCTV.설치연도 == 2021]

CCTV_2018_before = CCTV_2018_before.groupby("구").sum()
CCTV_2018_before = CCTV_2018_before.reset_index()
CCTV_2018_before = CCTV_2018_before.drop(["설치연도"], axis = 1)

CCTV_2019 = CCTV_2019.groupby("구").sum()
CCTV_2019 = CCTV_2019.reset_index()
CCTV_2019 = CCTV_2019.drop(["설치연도"], axis = 1)

CCTV_2020 = CCTV_2020.groupby("구").sum()
CCTV_2020 = CCTV_2020.reset_index()
CCTV_2020 = CCTV_2020.drop(["설치연도"], axis = 1)

CCTV_2021 = CCTV_2021.groupby("구").sum()
CCTV_2021 = CCTV_2021.reset_index()
CCTV_2021 = CCTV_2021.drop(["설치연도"], axis = 1)

CCTV_2019_Total = CCTV_2018_before.copy()

CCTV_2020_Total = pd.DataFrame()
CCTV_2020_Total["구"] = list(set(CCTV_2018_before.구.tolist())|set(CCTV_2019.구.tolist()))
CCTV_2020_Total[CCTV_2019.columns.drop(["구"])] = 0
CCTV_2020_Total = CCTV_2020_Total.apply(lambda x : df_add(x, CCTV_2018_before), axis = 1)
CCTV_2020_Total = CCTV_2020_Total.apply(lambda x : df_add(x, CCTV_2019), axis = 1)

CCTV_2021_Total = pd.DataFrame()
CCTV_2021_Total["구"] = list(set(CCTV_2020_Total.구.tolist())|set(CCTV_2020.구.tolist()))
CCTV_2021_Total[CCTV_2020.columns.drop(["구"])] = 0
CCTV_2021_Total = CCTV_2021_Total.apply(lambda x : df_add(x, CCTV_2020_Total), axis = 1)
CCTV_2021_Total = CCTV_2021_Total.apply(lambda x : df_add(x, CCTV_2020), axis = 1)

CCTV_2022_Total = pd.DataFrame()
CCTV_2022_Total["구"] = list(set(CCTV_2021_Total.구.tolist())|set(CCTV_2021.구.tolist()))
CCTV_2022_Total[CCTV_2021.columns.drop(["구"])] = 0
CCTV_2022_Total = CCTV_2022_Total.apply(lambda x : df_add(x, CCTV_2021_Total), axis = 1)
CCTV_2022_Total = CCTV_2022_Total.apply(lambda x : df_add(x, CCTV_2021), axis = 1)

CCTV_2019_Total["Year"] = "2019"
CCTV_2020_Total["Year"] = "2020"
CCTV_2021_Total["Year"] = "2021"
CCTV_2022_Total["Year"] = "2022"
CCTV_Total = pd.concat([CCTV_2019_Total, CCTV_2020_Total, CCTV_2021_Total, CCTV_2022_Total])

train = pd.merge(train, CCTV_Total, how = "left", on = ["구", "Year"])
train = train.fillna(0)
train["총CCTV개수"] = train[['속도단속', '신호단속', '불법주정차', '기타단속']].sum(axis=1)
test = pd.merge(test, CCTV_Total, how = "left", on = ["구", "Year"])
test = test.fillna(0)
test["총CCTV개수"] = test[['속도단속', '신호단속', '불법주정차', '기타단속']].sum(axis=1)

# 보안등 데이터 가공
light = light[[len(i.split())>=3 for i in light.소재지지번주소]]
light["구"] = [i.split()[2] for i in light.소재지지번주소]

light.구 = [re.sub(r"동[0-9]+-[0-9]+$|[0-9]동$|[0-9].[0-9]동$", "동", i) for i in light.구]
light.구 = [re.sub(r"[0-9]+-[0-9]+$", "", i) for i in light.구]
light.구 = [re.sub(r"[0-9]+$", "", i) for i in light.구]
light = light.iloc[[i for i in range(len(light)) if light.구.iloc[i] != ""]]
light = light.groupby("구").sum().설치개수.reset_index()
light = light.rename(columns = {"설치개수" : "보안등개수"})

train = pd.merge(train, light, how = "left", on = ["구"])
test = pd.merge(test, light, how = "left", on = ["구"])

# 어린이보호구역 데이터 가공
child["구"] = [i.split() if type(i) == str else i for i in child.소재지지번주소]
child["구"] = [i[0].split("?") if type(i) == list and len(i) == 2 else i for i in child.구]
child["구"] = [i[2] if type(i) == list else False for i in child.구]

for i in child[child.구 == False].index:
    child.구.iloc[i] = LOCAL.geo_coord2address(child.iloc[i].경도, child.iloc[i].위도, dataframe=True).address[0]["region_3depth_name"]
    
child.CCTV설치여부 = [1 if i == "Y" else 0 for i in child.CCTV설치여부]

child = child[["CCTV설치여부","구"]]
child = child.rename(columns = {"CCTC설치여부" : "어린이보호구역CCTV여부"})
child = child.groupby("구").sum().reset_index()

train = pd.merge(train, child, how = "left", on = ["구"])
train = train.fillna(0)
test = pd.merge(test, child, how = "left", on = ["구"])
test = test.fillna(0)

# 휴일 변수 추가
holidays = pd.DataFrame()
years = [2019, 2020, 2021]
for year in years:
    lst = pytimekr.holidays(year = year)
    for j in range(len(lst)):
        lst[j] = str(lst[j])
        lst[j] = lst[j].split("-")
    holidays = holidays.append(pd.DataFrame(lst, columns = ["Year","Month","Day"]))
holidays["휴일"] = 1

train = pd.merge(train, holidays, on = ["Year", "Month", "Day"], how = "left")
train = train.fillna(0)
train.휴일 = [1 if train.요일.iloc[i] == "토요일" or train.요일.iloc[i] == "일요일" else train.휴일.iloc[i] for i in range(len(train))]

holidays = pd.DataFrame([str(i).split("-") for i in pytimekr.holidays(year = 2022)], columns = ["Year","Month","Day"])
holidays["휴일"] = 1
test = pd.merge(test, holidays, on = ["Year", "Month", "Day"], how = "left")
test = test.fillna(0)
test.휴일 = [1 if test.요일.iloc[i] == "토요일" or test.요일.iloc[i] == "일요일" else test.휴일.iloc[i] for i in range(len(test))]

# 범주형 변수 전부 라벨링
category = ['요일', '기상상태', '노면상태', '사고유형', '군', '구', '도로형태대분류', '도로형태소분류']
for column in catagory:
    le = LabelEncoder()
    le.fit(train[column])
    train[column] = le.transform(train[column])
    test[column] = le.transform(test[column])

le.fit(train["Year_Month"].append(test["Year_Month"]))
train.Year_Month = le.transform(train.Year_Month)
test.Year_Month = le.transform(test.Year_Month)

for column in ["Hour","Year","Month","Day"]:
    train[column] = [int(i) for i in train[column]]
    test[column] = [int(i) for i in test[column]]
    
# train data X, Y 분리
x_train = train.drop(["사망자수", "중상자수","경상자수", "부상자수", "ECLO"], axis = 1)
y_train = train[["사망자수", "중상자수","경상자수", "부상자수", "ECLO"]]

# Min-Max Scaling
numeric = x_train.columns.drop(category)
scaler = MinMaxScaler()
scaler.fit(x_train[numeric])
x_train[numeric] = pd.DataFrame(scaler.transform(x_train[numeric]), columns = numeric)
test[numeric] = pd.DataFrame(scaler.transform(test[numeric]), columns = numeric)

# train, test 분리
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size = 0.3, shuffle = False, random_state = 1234)


In [99]:
cat_X_train = cb.Pool(X_train, label = Y_train.ECLO, cat_features = category)

catboost = cb.CatBoostRegressor()
catboost.fit(cat_X_train)

Learning rate set to 0.069208
0:	learn: 3.2614582	total: 35.3ms	remaining: 35.3s
1:	learn: 3.2560192	total: 71.4ms	remaining: 35.6s
2:	learn: 3.2515077	total: 106ms	remaining: 35.2s
3:	learn: 3.2478101	total: 132ms	remaining: 32.8s
4:	learn: 3.2440079	total: 165ms	remaining: 32.8s
5:	learn: 3.2404358	total: 194ms	remaining: 32.1s
6:	learn: 3.2381820	total: 213ms	remaining: 30.2s
7:	learn: 3.2354689	total: 244ms	remaining: 30.3s
8:	learn: 3.2331474	total: 277ms	remaining: 30.6s
9:	learn: 3.2309122	total: 310ms	remaining: 30.7s
10:	learn: 3.2298374	total: 323ms	remaining: 29s
11:	learn: 3.2271836	total: 352ms	remaining: 29s
12:	learn: 3.2254064	total: 386ms	remaining: 29.3s
13:	learn: 3.2235445	total: 419ms	remaining: 29.5s
14:	learn: 3.2223136	total: 455ms	remaining: 29.9s
15:	learn: 3.2214301	total: 485ms	remaining: 29.8s
16:	learn: 3.2206057	total: 506ms	remaining: 29.2s
17:	learn: 3.2194712	total: 536ms	remaining: 29.2s
18:	learn: 3.2186786	total: 564ms	remaining: 29.1s
19:	learn: 3.

In [103]:
cat_X_test = cb.Pool(data = X_test, cat_features = category)
pred = catboost.predict(cat_X_test)

In [111]:
cat_test = cb.Pool(data = test, cat_features = category)
pred = catboost.predict(cat_test)

In [118]:
sample = pd.read_csv("./data/sample_submission.csv")
sample.ECLO = pred
sample.to_csv("./data/result.csv", index= False)